In [6]:
import torch
from torch import nn
import kaggle
from pathlib import Path
import os
import numpy as np
import pandas as pd

In [7]:
csv_file_path = Path.cwd()/'dataset'/'breastCancer'

if csv_file_path.exists():
    print(f"Folder already exists at: {csv_file_path}")
else:
    csv_file_path.mkdir(parents=True, exist_ok=True)
    kaggle.api.dataset_download_files("yasserh/breast-cancer-dataset", path=csv_file_path, unzip=True)

Folder already exists at: c:\MLProject\DiseaseAndCancerTracker\Models\dataset\breastCancer


In [8]:
df = pd.read_csv(csv_file_path/'Breast_Cancer.csv')
df.head()

,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,68,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,4,Positive,Positive,24,1,60,Alive
1,50,White,Married,T2,N2,IIIA,Moderately differentiated,2,Regional,35,Positive,Positive,14,5,62,Alive
2,58,White,Divorced,T3,N3,IIIC,Moderately differentiated,2,Regional,63,Positive,Positive,14,7,75,Alive
3,58,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,18,Positive,Positive,2,1,84,Alive
4,47,White,Married,T2,N1,IIB,Poorly differentiated,3,Regional,41,Positive,Positive,3,1,50,Alive


In [9]:
value_counts = {col: df[col].value_counts() for col in df.columns}
for col, counts in value_counts.items():
    print(f"Value counts for {col}:  {counts}")
    print()

Value counts for Age:  Age
46    163
49    162
47    161
53    155
55    154
63    153
62    149
51    148
59    147
50    144
57    144
61    143
60    140
48    140
56    132
54    128
65    122
58    120
67    119
66    119
52    118
64    116
68    111
69    108
45    102
43     92
42     82
44     79
40     72
41     71
39     52
38     45
37     28
36     24
35     20
33     15
31     14
34     14
32     13
30      5
Name: count, dtype: int64

Value counts for Race:  Race
White    3413
Other     320
Black     291
Name: count, dtype: int64

Value counts for Marital Status:  Marital Status
Married      2643
Single        615
Divorced      486
Widowed       235
Separated      45
Name: count, dtype: int64

Value counts for T Stage :  T Stage 
T2    1786
T1    1603
T3     533
T4     102
Name: count, dtype: int64

Value counts for N Stage:  N Stage
N1    2732
N2     820
N3     472
Name: count, dtype: int64

Value counts for 6th Stage:  6th Stage
IIA     1305
IIB     1130
IIIA    1050
I